In [1]:
%load_ext rpy2.ipython

In [8]:
%%R
library(dplyr)
library(magrittr)
library(ggplot2)
library(tnet)

In [42]:
%%R
data<-read.table("./data/all_info.txt", sep="\t", header=T)
#str(data)

data$bray_clusters_eps0.9[is.na(data$bray_clusters_eps0.9)]<-0
data$bray_clusters_eps0.9[data$bray_clusters_eps0.9==0]<-as.character(data$phage[data$bray_clusters_eps0.9==0])
#data$phage[data$bray_clusters_eps0.9==0]%>%print
#as.character(data$phage[data$bray_clsuters_eps0.9==0])
data$bray_clusters_eps0.9%>%as.character%>%as.factor->data$bray_clusters_eps0.9

adj<-read.table("./data/phagevphage_orthomcl.adjacency.txt", header=TRUE, sep="\t")
head(adj)

adj$dbgrp<-data$bray_clusters_eps0.9[match(adj$organism, data$phage)]
adj%>%group_by(protein_cluster, dbgrp)%>%summarise(count=n())->grp_pclusts
print(grp_pclusts%>%head)
write.table(grp_pclusts[,c(1,2)], file="./data/bray_eps0.9_group_protclust_adj.txt", quote=FALSE, row.names=FALSE, sep="\t")

Source: local data frame [6 x 3]
Groups: protein_cluster [2]

  protein_cluster    dbgrp count
           (fctr)   (fctr) (int)
1         pvp1000       13    24
2         pvp1000       15     8
3         pvp1000       19    34
4         pvp1000       27     6
5         pvp1000        6    20
6         pvp1001 1.029.O.     1


In [43]:
import pandas as pd
import numpy as np
from pandas import DataFrame, Series

def create_mat_from_edgetable(edgetable, output, ind_cat, col_cat):
    adj=pd.read_table(edgetable).drop_duplicates()
    adj['pos']=1
    mat=adj.pivot_table(values='pos', index=ind_cat, columns=col_cat)
    mat=mat.fillna(0)
    mat.applymap(lambda x: x.astype(int)).to_csv(output,sep="\t")
    
create_mat_from_edgetable("./data/bray_eps0.9_group_protclust_adj.txt","./data/bray_eps0.9_group_protclust_mat.txt", "dbgrp","protein_cluster")

In [55]:
%%R 
mat=read.table("./data/bray_eps0.9_group_protclust_mat.txt", sep="\t", header=TRUE)


rownames(mat)<-as.factor(mat$dbgrp)
mat$dbgrp<-NULL



net<-as.tnet(mat, type="binary two-mode tnet")
proj1<-projecting_tm(net, method="sum")

proj1$i%>%unique%>%length


groupnames<-as.character(row.names(mat1))
number<-as.character(c(1:length(groupnames)))
groupid<-data.frame(cbind(groupnames, number))

proj1$groupi<-groupid$groupnames[match(proj1$i, groupid$number)]
proj1$groupj<-groupid$groupnames[match(proj1$j, groupid$number)]

proj1
write.table(proj1[,c(4,5,3)], "./data/bray_groups_weighted_one_mode.txt", sep="\t", quote=FALSE, row.names=FALSE, col.names=TRUE)

In [49]:
def dereplicate_edgelist(input_file, outfile):
    pairs=[]
    out=open(outfile,"w")
    with open(input_file) as infile:
        for l in infile:
            i=l.split("\t")[0]
            j=l.split("\t")[1]
            if (i,j) not in pairs and (j,i) not in pairs:
                pairs.append((i,j))
                out.write(l)
    out.close()

dereplicate_edgelist("./data/bray_groups_weighted_one_mode.txt", "./data/bray_groups_weighted_one_mode_derep.txt")

In [50]:
create_mat_from_edgetable("./data/phagevphage_orthomcl.adjacency.txt","./data/phagevphage_orthomcl.matrix.txt", "organism", "protein_cluster")

In [56]:
%%R 
mat=read.table("./data/phagevphage_orthomcl.matrix.txt", sep="\t", header=TRUE)


rownames(mat)<-as.factor(mat$organism)
mat$organism<-NULL



net<-as.tnet(mat, type="binary two-mode tnet")
proj1<-projecting_tm(net, method="sum")

proj1$i%>%unique%>%length


groupnames<-as.character(row.names(mat))
number<-as.character(c(1:length(groupnames)))
groupid<-data.frame(cbind(groupnames, number))

proj1$groupi<-groupid$groupnames[match(proj1$i, groupid$number)]
proj1$groupj<-groupid$groupnames[match(proj1$j, groupid$number)]

proj1
write.table(proj1[,c(4,5,3)], "./data/indiv_phages_weighted_one_mode.txt", sep="\t", quote=FALSE, row.names=FALSE, col.names=TRUE)

In [57]:
dereplicate_edgelist("./data/indiv_phages_weighted_one_mode.txt","./data/indiv_phages_weighted_one_mode_derep.txt")